In [ ]:
images=[]
labels=[]

In [ ]:
import os
import cv2
import numpy as np

In [ ]:
for i in ['without_mask', 'with_mask']:
    imageNameList=os.listdir("D:AI/" + str(i))
    for fileName in imageNameList:
        Image=cv2.imread("D:AI/" + str(i) + "/" + str(fileName))
        try:
            gray=cv2.cvtColor(Image,cv2.COLOR_BGR2GRAY)
            Image=cv2.resize(gray,(100,100))
            images.append(Image)
            if i=="without_mask":
                labels.append(0)
            else:
                labels.append(1)
        except:
            pass
    print("Inside folder ", i)

In [ ]:
images=np.array(images)
labels=np.array(labels)

In [ ]:
def preprocessing(img):
    img=img/255
    return img

In [ ]:
images=images/255.0
images=np.reshape(images,(images.shape[0],100,100,1))

In [ ]:
from sklearn.model_selection import train_test_split
train_features,test_features,train_target,test_target=train_test_split(images,labels,test_size=0.2)
train_features=train_features.reshape(train_features.shape[0], train_features.shape[1],train_features.shape[2],1)
test_features=test_features.reshape(test_features.shape[0],test_features.shape[1],test_features.shape[2],1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
dataGenerator=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1,rotation_range=10)
dataGenerator.fit(train_features)
batches=dataGenerator.flow(train_features,train_target,batch_size=20)


In [ ]:
from keras.utils import to_categorical
train_target=to_categorical(train_target)
test_target=to_categorical(test_target)
from keras.models import Sequential
from keras.layers import Flatten,Conv2D,MaxPooling2D,Dense,Dropout

In [ ]:
model=Sequential()
model.add(Conv2D(64,(3,3),activation="relu",input_shape=(100, 100, 1)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))

In [ ]:
from keras.optimizers import Adam
model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
fitted_model=model.fit_generator(dataGenerator.flow(train_features,train_target,batch_size=20),epochs=30)
model.save('Face_Mask_Detector.h5')

In [ ]:
from keras.models import load_model
model=load_model('Face_Mask_Detector.h5')

In [ ]:
def getClassName(classNo):
    if   classNo == 0: return 'without_mask'
    elif classNo == 1: return 'with_mask'

In [ ]:
cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
cap.set(10,180)

In [ ]:
face_cascade = cv2.CascadeClassifier('D:/frontalFace10/haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    ret,image=cap.read()
    grayscale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(grayscale, 1.1, 5)
    for (x,y,w,h) in faces:
        face_img=grayscale[y:y+h,x:x+w]
        resised_img=cv2.resize(face_img,(100,100))
        normalised_img=preprocessing(resised_img)
        reshaped_img=np.reshape(normalised_img,(1,100,100,1))
        result=model.predict(reshaped_img)
        
        classIndex=np.argmax(result,axis=1)[0]
        probability=np.amax(result)
        if classIndex==0:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(image,getClassName(classIndex),(x,y-10),font,1,(0,0,255),2)
            cv2.imshow("RESULT",image)
        else:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(image,getClassName(classIndex),(x,y-10),font,1,(0,255,0),2)
            cv2.imshow("RESULT",image)
        key=cv2.waitKey(1)
        if key==27:
            cv2.destroyAllWindows()
            cap.release()
            break
        
  